In [1]:
import pandas as pd
import numpy as np
import neurokit2 as nk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
low=pd.read_csv('gsr_low.csv')
low

,Unnamed: 0,Trial 1:0back,Trial 6:0back
0,0,4.038,6.964
1,1,4.040,6.967
2,2,4.041,6.969
3,3,4.042,6.971
4,4,4.043,6.972
...,...,...,...
1689595,1689595,0.220,0.485
1689596,1689596,0.220,0.485
1689597,1689597,0.220,0.485
1689598,1689598,0.220,0.485


In [3]:
high=pd.read_csv('gsr_high.csv')

In [4]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
dfl=sc.fit_transform(low)
dfh=sc.fit_transform(high)

In [5]:
dfl=pd.DataFrame(dfl)
dfl

,0,1,2
0,-1.732050,0.729059,1.530305
1,-1.732048,0.729808,1.531406
2,-1.732046,0.730183,1.532139
3,-1.732044,0.730558,1.532873
4,-1.732042,0.730933,1.533240
...,...,...,...
1689595,1.732042,-0.702171,-0.846970
1689596,1.732044,-0.702171,-0.846970
1689597,1.732046,-0.702171,-0.846970
1689598,1.732048,-0.702171,-0.846970


In [6]:
dfh=pd.DataFrame(dfh)
dfh

,0,1,2
0,-1.732050,1.294932,1.227664
1,-1.732048,1.294599,1.227306
2,-1.732046,1.294599,1.227664
3,-1.732044,1.294599,1.227664
4,-1.732042,1.294599,1.227306
...,...,...,...
1689595,1.732042,-0.822600,-0.941763
1689596,1.732044,-0.821934,-0.942120
1689597,1.732046,-0.822267,-0.942120
1689598,1.732048,-0.822267,-0.942120


In [7]:
dfl=dfl.iloc[:,1:3]
dfh=dfh.iloc[:,1:3]
dfl


,1,2
0,0.729059,1.530305
1,0.729808,1.531406
2,0.730183,1.532139
3,0.730558,1.532873
4,0.730933,1.533240
...,...,...
1689595,-0.702171,-0.846970
1689596,-0.702171,-0.846970
1689597,-0.702171,-0.846970
1689598,-0.702171,-0.846970


In [8]:
from scipy.stats import skew, kurtosis 

In [9]:
import numpy as np
import pandas as pd
import neurokit2 as nk
from scipy import stats
from scipy.signal import find_peaks


features = {'mean': [],'std': [],'range': [],'slope': [],'skew': [],'kurtosis': [],'heart_rate': [],'peak_amplitude': [],'num_peaks': [],'AUC': [],'IQR': [],'CV': [], 'SDNN': [],'RMSSD': []}

sampling_rate = 256  
for i in range(dfl.shape[1]):  
    for j in range(0, len(dfl), 768):  
        set_val = dfl.iloc[j:j + 768, i]
        if len(set_val) < 768:
            continue  
        signal = set_val.values
        filtered_signal = nk.signal_filter(signal, sampling_rate=sampling_rate, lowcut=0.5, highcut=10)
        time = np.arange(len(filtered_signal)) / sampling_rate
        features['mean'].append(np.mean(filtered_signal))
        features['std'].append(np.std(filtered_signal))
        peaks, _ = find_peaks(filtered_signal, height=0)  
        features['num_peaks'].append(len(peaks))
        if len(peaks) > 1:
            peak_times = time[peaks]
            hr = 60 / np.mean(np.diff(peak_times))  
            features['heart_rate'].append(hr)

           
            rr_intervals = np.diff(peak_times)
            features['SDNN'].append(np.std(rr_intervals) if len(rr_intervals) > 1 else 0)
            features['RMSSD'].append(np.sqrt(np.mean(np.square(np.diff(rr_intervals)))) if len(rr_intervals) > 1 else 0)
        else:
            features['heart_rate'].append(0) 
            features['SDNN'].append(0)
            features['RMSSD'].append(0)

        
        if peaks.size > 0:
            features['peak_amplitude'].append(np.max(filtered_signal[peaks]))
        else:
            features['peak_amplitude'].append(0)

        
        auc = np.trapz(filtered_signal, time)
        features['AUC'].append(auc)

      
        features['range'].append(np.max(filtered_signal) - np.min(filtered_signal))
        features['slope'].append(np.polyfit(time, filtered_signal, 1)[0])  
        features['skew'].append(stats.skew(filtered_signal))
        features['kurtosis'].append(stats.kurtosis(filtered_signal))
        features['IQR'].append(np.percentile(filtered_signal, 75) - np.percentile(filtered_signal, 25))
        features['CV'].append(np.std(filtered_signal) / np.mean(filtered_signal) if np.mean(filtered_signal) != 0 else 0)


features_df = pd.DataFrame(features)

In [10]:
features_df

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD
0,0.000996,0.004718,0.020617,-0.002308,0.661511,-0.093439,50.032573,0.011768,3,0.003002,0.004619,4.738685,0.183594,0.367188
1,0.000476,0.002138,0.007271,-0.001150,-0.019341,-1.010784,46.616085,0.004252,3,0.001429,0.003153,4.493457,0.427734,0.855469
2,-0.000397,0.004629,0.015468,-0.000022,-0.447256,-1.105105,92.160000,0.006090,4,-0.001201,0.007734,-11.649189,0.447208,0.564625
3,0.000640,0.005965,0.019192,-0.000779,-0.350907,-1.230257,60.591716,0.009231,3,0.001944,0.010999,9.322012,0.130859,0.261719
4,-0.000577,0.007073,0.024573,0.001057,0.788647,-0.510958,52.067797,0.015160,2,-0.001729,0.010224,-12.264147,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.000207,0.001770,0.009029,-0.000409,1.148990,1.322124,187.571235,0.006650,10,0.000619,0.002210,8.560249,0.143778,0.260004
4396,-0.000084,0.001934,0.010688,0.000220,1.167008,1.823354,261.224490,0.007069,11,-0.000254,0.002055,-23.049100,0.142295,0.167999
4397,-0.000123,0.001562,0.007390,0.000237,0.755971,0.450115,187.889908,0.004395,9,-0.000370,0.001474,-12.675818,0.334371,0.560193
4398,0.000479,0.002237,0.010475,-0.000817,0.693602,0.147952,264.000000,0.006921,12,0.001436,0.002969,4.669963,0.315890,0.484413


In [11]:
import numpy as np
import pandas as pd
import neurokit2 as nk
from scipy import stats
from scipy.signal import find_peaks


features = {'mean': [],'std': [],'range': [],'slope': [],'skew': [],'kurtosis': [],'heart_rate': [],'peak_amplitude': [],'num_peaks': [],'AUC': [],'IQR': [],'CV': [], 'SDNN': [],'RMSSD': []}


sampling_rate = 256 

for i in range(dfh.shape[1]):  
    for j in range(0, len(dfh), 768):  
        set_val = dfh.iloc[j:j + 768, i]
        if len(set_val) < 768:
            continue  

       
        signal = set_val.values

       
        filtered_signal = nk.signal_filter(signal, sampling_rate=sampling_rate, lowcut=0.5, highcut=10)

        
        time = np.arange(len(filtered_signal)) / sampling_rate
        
       
        features['mean'].append(np.mean(filtered_signal))
        features['std'].append(np.std(filtered_signal))

       
        peaks, _ = find_peaks(filtered_signal, height=0)  
        features['num_peaks'].append(len(peaks))
        
       
        if len(peaks) > 1:
            peak_times = time[peaks]
            hr = 60 / np.mean(np.diff(peak_times))  # In beats per minute
            features['heart_rate'].append(hr)

           
            rr_intervals = np.diff(peak_times)
            features['SDNN'].append(np.std(rr_intervals) if len(rr_intervals) > 1 else 0)
            features['RMSSD'].append(np.sqrt(np.mean(np.square(np.diff(rr_intervals)))) if len(rr_intervals) > 1 else 0)
        else:
            features['heart_rate'].append(0) 
            features['SDNN'].append(0)
            features['RMSSD'].append(0)

        # Peak Amplitude
        if peaks.size > 0:
            features['peak_amplitude'].append(np.max(filtered_signal[peaks]))
        else:
            features['peak_amplitude'].append(0)

       
        auc = np.trapz(filtered_signal, time)
        features['AUC'].append(auc)

        # Range, slope, skew and kurtosis
        features['range'].append(np.max(filtered_signal) - np.min(filtered_signal))
        features['slope'].append(np.polyfit(time, filtered_signal, 1)[0])  
        features['skew'].append(stats.skew(filtered_signal))
        features['kurtosis'].append(stats.kurtosis(filtered_signal))
        features['IQR'].append(np.percentile(filtered_signal, 75) - np.percentile(filtered_signal, 25))
        features['CV'].append(np.std(filtered_signal) / np.mean(filtered_signal) if np.mean(filtered_signal) != 0 else 0)
features_dfh = pd.DataFrame(features)

In [12]:
features_dfh

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD
0,-0.000223,0.006353,0.022128,0.000785,-0.519752,-0.696261,32.542373,0.009081,2,-0.000677,0.007771,-28.496445,0.000000,0.000000
1,-0.000334,0.006147,0.021371,0.001301,0.046888,-0.995699,44.651163,0.011187,2,-0.000995,0.010210,-18.405320,0.000000,0.000000
2,-0.000042,0.009064,0.032232,-0.001181,-0.648272,-0.380301,42.905028,0.012025,2,-0.000133,0.011983,-214.691931,0.000000,0.000000
3,0.001281,0.005804,0.031798,-0.002337,0.118839,0.365827,75.294118,0.012405,3,0.003881,0.006670,4.532513,0.597656,1.195312
4,0.000658,0.004404,0.018659,-0.001272,-0.226269,-0.865081,62.950820,0.007204,3,0.001995,0.008092,6.693004,0.886719,1.773438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.000031,0.001292,0.005908,0.000030,0.586278,-0.242693,349.558233,0.003534,18,0.000094,0.001744,41.633710,0.086515,0.129110
4396,-0.000017,0.001079,0.005346,0.000089,0.293458,-0.273892,346.629055,0.003053,17,-0.000050,0.001451,-62.713875,0.089137,0.140683
4397,0.000147,0.001260,0.005657,-0.000310,0.549715,-0.350928,429.507324,0.003440,22,0.000438,0.001722,8.593761,0.059154,0.085986
4398,-0.000105,0.001046,0.004942,0.000258,-0.053679,-0.487556,414.016173,0.002551,21,-0.000318,0.001456,-9.927584,0.107914,0.143048


In [13]:
features_df

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD
0,0.000996,0.004718,0.020617,-0.002308,0.661511,-0.093439,50.032573,0.011768,3,0.003002,0.004619,4.738685,0.183594,0.367188
1,0.000476,0.002138,0.007271,-0.001150,-0.019341,-1.010784,46.616085,0.004252,3,0.001429,0.003153,4.493457,0.427734,0.855469
2,-0.000397,0.004629,0.015468,-0.000022,-0.447256,-1.105105,92.160000,0.006090,4,-0.001201,0.007734,-11.649189,0.447208,0.564625
3,0.000640,0.005965,0.019192,-0.000779,-0.350907,-1.230257,60.591716,0.009231,3,0.001944,0.010999,9.322012,0.130859,0.261719
4,-0.000577,0.007073,0.024573,0.001057,0.788647,-0.510958,52.067797,0.015160,2,-0.001729,0.010224,-12.264147,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.000207,0.001770,0.009029,-0.000409,1.148990,1.322124,187.571235,0.006650,10,0.000619,0.002210,8.560249,0.143778,0.260004
4396,-0.000084,0.001934,0.010688,0.000220,1.167008,1.823354,261.224490,0.007069,11,-0.000254,0.002055,-23.049100,0.142295,0.167999
4397,-0.000123,0.001562,0.007390,0.000237,0.755971,0.450115,187.889908,0.004395,9,-0.000370,0.001474,-12.675818,0.334371,0.560193
4398,0.000479,0.002237,0.010475,-0.000817,0.693602,0.147952,264.000000,0.006921,12,0.001436,0.002969,4.669963,0.315890,0.484413


In [14]:
features_dfh

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD
0,-0.000223,0.006353,0.022128,0.000785,-0.519752,-0.696261,32.542373,0.009081,2,-0.000677,0.007771,-28.496445,0.000000,0.000000
1,-0.000334,0.006147,0.021371,0.001301,0.046888,-0.995699,44.651163,0.011187,2,-0.000995,0.010210,-18.405320,0.000000,0.000000
2,-0.000042,0.009064,0.032232,-0.001181,-0.648272,-0.380301,42.905028,0.012025,2,-0.000133,0.011983,-214.691931,0.000000,0.000000
3,0.001281,0.005804,0.031798,-0.002337,0.118839,0.365827,75.294118,0.012405,3,0.003881,0.006670,4.532513,0.597656,1.195312
4,0.000658,0.004404,0.018659,-0.001272,-0.226269,-0.865081,62.950820,0.007204,3,0.001995,0.008092,6.693004,0.886719,1.773438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.000031,0.001292,0.005908,0.000030,0.586278,-0.242693,349.558233,0.003534,18,0.000094,0.001744,41.633710,0.086515,0.129110
4396,-0.000017,0.001079,0.005346,0.000089,0.293458,-0.273892,346.629055,0.003053,17,-0.000050,0.001451,-62.713875,0.089137,0.140683
4397,0.000147,0.001260,0.005657,-0.000310,0.549715,-0.350928,429.507324,0.003440,22,0.000438,0.001722,8.593761,0.059154,0.085986
4398,-0.000105,0.001046,0.004942,0.000258,-0.053679,-0.487556,414.016173,0.002551,21,-0.000318,0.001456,-9.927584,0.107914,0.143048


In [15]:
import pywt

In [16]:

import pywt
fs=256
def extract_features(signal):
    sampling_freq = 256
    wavelet = 'db4'
    coeffs = pywt.wavedec(signal, wavelet, level=5)
    cA5, cD5, cD4, cD3, cD2, cD1 = coeffs
    vlf_power = np.sum(np.square(cA5))
    lf_power = np.sum(np.square(cD5))
    hf_power = np.sum(np.square(cD4))
    total_power = vlf_power + lf_power + hf_power
    lf_hf_ratio = lf_power / hf_power

    return {
        'VLF': vlf_power,
        'LF': lf_power,
        'HF': hf_power,
        'total_power': total_power,
        'lf/hf':lf_hf_ratio
         }

# Process data in chunks of 768 samples
results = []
i,j,k=0,1,0
while k<2:
    while j<2201:
        while i<768*j:
            set=dfl.iloc[i:i+768,k]
            ff=extract_features(set.values)
            i=i+768
            results.append(ff)
        j=j+1
    k=k+1
    j=0
    i=0
    


In [17]:
rs=pd.DataFrame(results)

In [18]:
rs

,VLF,LF,HF,total_power,lf/hf
0,472.230494,0.000047,0.000011,472.230552,4.193497
1,363.034653,0.000014,0.000003,363.034670,4.594817
2,290.521636,0.000107,0.000025,290.521768,4.285143
3,576.867403,0.000132,0.000026,576.867561,5.140528
4,815.306424,0.000055,0.000007,815.306487,7.505305
...,...,...,...,...,...
4395,682.347133,0.000436,0.000205,682.347774,2.132526
4396,684.264398,0.000571,0.000198,684.265166,2.888093
4397,686.898639,0.000124,0.000127,686.898889,0.976401
4398,685.513055,0.000317,0.000391,685.513763,0.809132


In [19]:
features_df

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD
0,0.000996,0.004718,0.020617,-0.002308,0.661511,-0.093439,50.032573,0.011768,3,0.003002,0.004619,4.738685,0.183594,0.367188
1,0.000476,0.002138,0.007271,-0.001150,-0.019341,-1.010784,46.616085,0.004252,3,0.001429,0.003153,4.493457,0.427734,0.855469
2,-0.000397,0.004629,0.015468,-0.000022,-0.447256,-1.105105,92.160000,0.006090,4,-0.001201,0.007734,-11.649189,0.447208,0.564625
3,0.000640,0.005965,0.019192,-0.000779,-0.350907,-1.230257,60.591716,0.009231,3,0.001944,0.010999,9.322012,0.130859,0.261719
4,-0.000577,0.007073,0.024573,0.001057,0.788647,-0.510958,52.067797,0.015160,2,-0.001729,0.010224,-12.264147,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.000207,0.001770,0.009029,-0.000409,1.148990,1.322124,187.571235,0.006650,10,0.000619,0.002210,8.560249,0.143778,0.260004
4396,-0.000084,0.001934,0.010688,0.000220,1.167008,1.823354,261.224490,0.007069,11,-0.000254,0.002055,-23.049100,0.142295,0.167999
4397,-0.000123,0.001562,0.007390,0.000237,0.755971,0.450115,187.889908,0.004395,9,-0.000370,0.001474,-12.675818,0.334371,0.560193
4398,0.000479,0.002237,0.010475,-0.000817,0.693602,0.147952,264.000000,0.006921,12,0.001436,0.002969,4.669963,0.315890,0.484413


In [20]:
low_data=pd.concat([features_df,rs],axis=1)

In [21]:
low_data

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD,VLF,LF,HF,total_power,lf/hf
0,0.000996,0.004718,0.020617,-0.002308,0.661511,-0.093439,50.032573,0.011768,3,0.003002,0.004619,4.738685,0.183594,0.367188,472.230494,0.000047,0.000011,472.230552,4.193497
1,0.000476,0.002138,0.007271,-0.001150,-0.019341,-1.010784,46.616085,0.004252,3,0.001429,0.003153,4.493457,0.427734,0.855469,363.034653,0.000014,0.000003,363.034670,4.594817
2,-0.000397,0.004629,0.015468,-0.000022,-0.447256,-1.105105,92.160000,0.006090,4,-0.001201,0.007734,-11.649189,0.447208,0.564625,290.521636,0.000107,0.000025,290.521768,4.285143
3,0.000640,0.005965,0.019192,-0.000779,-0.350907,-1.230257,60.591716,0.009231,3,0.001944,0.010999,9.322012,0.130859,0.261719,576.867403,0.000132,0.000026,576.867561,5.140528
4,-0.000577,0.007073,0.024573,0.001057,0.788647,-0.510958,52.067797,0.015160,2,-0.001729,0.010224,-12.264147,0.000000,0.000000,815.306424,0.000055,0.000007,815.306487,7.505305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.000207,0.001770,0.009029,-0.000409,1.148990,1.322124,187.571235,0.006650,10,0.000619,0.002210,8.560249,0.143778,0.260004,682.347133,0.000436,0.000205,682.347774,2.132526
4396,-0.000084,0.001934,0.010688,0.000220,1.167008,1.823354,261.224490,0.007069,11,-0.000254,0.002055,-23.049100,0.142295,0.167999,684.264398,0.000571,0.000198,684.265166,2.888093
4397,-0.000123,0.001562,0.007390,0.000237,0.755971,0.450115,187.889908,0.004395,9,-0.000370,0.001474,-12.675818,0.334371,0.560193,686.898639,0.000124,0.000127,686.898889,0.976401
4398,0.000479,0.002237,0.010475,-0.000817,0.693602,0.147952,264.000000,0.006921,12,0.001436,0.002969,4.669963,0.315890,0.484413,685.513055,0.000317,0.000391,685.513763,0.809132


In [22]:
low

,Unnamed: 0,Trial 1:0back,Trial 6:0back
0,0,4.038,6.964
1,1,4.040,6.967
2,2,4.041,6.969
3,3,4.042,6.971
4,4,4.043,6.972
...,...,...,...
1689595,1689595,0.220,0.485
1689596,1689596,0.220,0.485
1689597,1689597,0.220,0.485
1689598,1689598,0.220,0.485


In [23]:
fs=256
def extract_features(signal):
    sampling_freq = 256
    wavelet = 'db4'
    coeffs = pywt.wavedec(signal, wavelet, level=5)
    cA5, cD5, cD4, cD3, cD2, cD1 = coeffs
    vlf_power = np.sum(np.square(cA5))
    lf_power = np.sum(np.square(cD5))
    hf_power = np.sum(np.square(cD4))
    total_power = vlf_power + lf_power + hf_power
    lf_hf_ratio = lf_power / hf_power

    return {
        'VLF': vlf_power,
        'LF': lf_power,
        'HF': hf_power,
        'total_power': total_power,
        'lf/hf':lf_hf_ratio
         }

# Process data in chunks of 768 samples
results = []
i,j,k=0,1,0
while k<2:
    while j<2201:
        while i<768*j:
            set=dfh.iloc[i:i+768,k]
            ff=extract_features(set.values)
            i=i+768
            results.append(ff)
        j=j+1
    k=k+1
    j=0
    i=0
    

In [24]:
rsh=pd.DataFrame(results)

In [25]:
rsh

,VLF,LF,HF,total_power,lf/hf
0,1736.369227,0.000004,9.225929e-07,1736.369232,4.186629
1,2146.069834,0.000003,1.157572e-06,2146.069838,2.827218
2,2328.979869,0.000178,3.280511e-05,2328.980081,5.437959
3,3044.344808,0.000271,5.380093e-05,3044.345132,5.028233
4,3340.914220,0.000054,1.049859e-05,3340.914285,5.187004
...,...,...,...,...,...
4395,844.471080,0.000329,4.389741e-04,844.471848,0.749276
4396,846.049114,0.000279,4.588363e-04,846.049852,0.608788
4397,845.711391,0.000299,5.538104e-04,845.712244,0.539416
4398,848.368769,0.000164,5.148420e-04,848.369448,0.319224


In [26]:
high_data=pd.concat([features_dfh,rsh],axis=1)

In [27]:
high_data

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD,VLF,LF,HF,total_power,lf/hf
0,-0.000223,0.006353,0.022128,0.000785,-0.519752,-0.696261,32.542373,0.009081,2,-0.000677,0.007771,-28.496445,0.000000,0.000000,1736.369227,0.000004,9.225929e-07,1736.369232,4.186629
1,-0.000334,0.006147,0.021371,0.001301,0.046888,-0.995699,44.651163,0.011187,2,-0.000995,0.010210,-18.405320,0.000000,0.000000,2146.069834,0.000003,1.157572e-06,2146.069838,2.827218
2,-0.000042,0.009064,0.032232,-0.001181,-0.648272,-0.380301,42.905028,0.012025,2,-0.000133,0.011983,-214.691931,0.000000,0.000000,2328.979869,0.000178,3.280511e-05,2328.980081,5.437959
3,0.001281,0.005804,0.031798,-0.002337,0.118839,0.365827,75.294118,0.012405,3,0.003881,0.006670,4.532513,0.597656,1.195312,3044.344808,0.000271,5.380093e-05,3044.345132,5.028233
4,0.000658,0.004404,0.018659,-0.001272,-0.226269,-0.865081,62.950820,0.007204,3,0.001995,0.008092,6.693004,0.886719,1.773438,3340.914220,0.000054,1.049859e-05,3340.914285,5.187004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.000031,0.001292,0.005908,0.000030,0.586278,-0.242693,349.558233,0.003534,18,0.000094,0.001744,41.633710,0.086515,0.129110,844.471080,0.000329,4.389741e-04,844.471848,0.749276
4396,-0.000017,0.001079,0.005346,0.000089,0.293458,-0.273892,346.629055,0.003053,17,-0.000050,0.001451,-62.713875,0.089137,0.140683,846.049114,0.000279,4.588363e-04,846.049852,0.608788
4397,0.000147,0.001260,0.005657,-0.000310,0.549715,-0.350928,429.507324,0.003440,22,0.000438,0.001722,8.593761,0.059154,0.085986,845.711391,0.000299,5.538104e-04,845.712244,0.539416
4398,-0.000105,0.001046,0.004942,0.000258,-0.053679,-0.487556,414.016173,0.002551,21,-0.000318,0.001456,-9.927584,0.107914,0.143048,848.368769,0.000164,5.148420e-04,848.369448,0.319224


In [28]:
low_data['label']=0
high_data['label']=1

In [29]:
data=pd.concat([low_data,high_data],ignore_index=True)
data

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD,VLF,LF,HF,total_power,lf/hf,label
0,0.000996,0.004718,0.020617,-0.002308,0.661511,-0.093439,50.032573,0.011768,3,0.003002,0.004619,4.738685,0.183594,0.367188,472.230494,0.000047,0.000011,472.230552,4.193497,0
1,0.000476,0.002138,0.007271,-0.001150,-0.019341,-1.010784,46.616085,0.004252,3,0.001429,0.003153,4.493457,0.427734,0.855469,363.034653,0.000014,0.000003,363.034670,4.594817,0
2,-0.000397,0.004629,0.015468,-0.000022,-0.447256,-1.105105,92.160000,0.006090,4,-0.001201,0.007734,-11.649189,0.447208,0.564625,290.521636,0.000107,0.000025,290.521768,4.285143,0
3,0.000640,0.005965,0.019192,-0.000779,-0.350907,-1.230257,60.591716,0.009231,3,0.001944,0.010999,9.322012,0.130859,0.261719,576.867403,0.000132,0.000026,576.867561,5.140528,0
4,-0.000577,0.007073,0.024573,0.001057,0.788647,-0.510958,52.067797,0.015160,2,-0.001729,0.010224,-12.264147,0.000000,0.000000,815.306424,0.000055,0.000007,815.306487,7.505305,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,0.000031,0.001292,0.005908,0.000030,0.586278,-0.242693,349.558233,0.003534,18,0.000094,0.001744,41.633710,0.086515,0.129110,844.471080,0.000329,0.000439,844.471848,0.749276,1
8796,-0.000017,0.001079,0.005346,0.000089,0.293458,-0.273892,346.629055,0.003053,17,-0.000050,0.001451,-62.713875,0.089137,0.140683,846.049114,0.000279,0.000459,846.049852,0.608788,1
8797,0.000147,0.001260,0.005657,-0.000310,0.549715,-0.350928,429.507324,0.003440,22,0.000438,0.001722,8.593761,0.059154,0.085986,845.711391,0.000299,0.000554,845.712244,0.539416,1
8798,-0.000105,0.001046,0.004942,0.000258,-0.053679,-0.487556,414.016173,0.002551,21,-0.000318,0.001456,-9.927584,0.107914,0.143048,848.368769,0.000164,0.000515,848.369448,0.319224,1


In [80]:
data.to_csv('gsr_feature.csv')

In [30]:
x=data.iloc[:,0:19]

In [31]:
x

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD,VLF,LF,HF,total_power,lf/hf
0,0.000996,0.004718,0.020617,-0.002308,0.661511,-0.093439,50.032573,0.011768,3,0.003002,0.004619,4.738685,0.183594,0.367188,472.230494,0.000047,0.000011,472.230552,4.193497
1,0.000476,0.002138,0.007271,-0.001150,-0.019341,-1.010784,46.616085,0.004252,3,0.001429,0.003153,4.493457,0.427734,0.855469,363.034653,0.000014,0.000003,363.034670,4.594817
2,-0.000397,0.004629,0.015468,-0.000022,-0.447256,-1.105105,92.160000,0.006090,4,-0.001201,0.007734,-11.649189,0.447208,0.564625,290.521636,0.000107,0.000025,290.521768,4.285143
3,0.000640,0.005965,0.019192,-0.000779,-0.350907,-1.230257,60.591716,0.009231,3,0.001944,0.010999,9.322012,0.130859,0.261719,576.867403,0.000132,0.000026,576.867561,5.140528
4,-0.000577,0.007073,0.024573,0.001057,0.788647,-0.510958,52.067797,0.015160,2,-0.001729,0.010224,-12.264147,0.000000,0.000000,815.306424,0.000055,0.000007,815.306487,7.505305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,0.000031,0.001292,0.005908,0.000030,0.586278,-0.242693,349.558233,0.003534,18,0.000094,0.001744,41.633710,0.086515,0.129110,844.471080,0.000329,0.000439,844.471848,0.749276
8796,-0.000017,0.001079,0.005346,0.000089,0.293458,-0.273892,346.629055,0.003053,17,-0.000050,0.001451,-62.713875,0.089137,0.140683,846.049114,0.000279,0.000459,846.049852,0.608788
8797,0.000147,0.001260,0.005657,-0.000310,0.549715,-0.350928,429.507324,0.003440,22,0.000438,0.001722,8.593761,0.059154,0.085986,845.711391,0.000299,0.000554,845.712244,0.539416
8798,-0.000105,0.001046,0.004942,0.000258,-0.053679,-0.487556,414.016173,0.002551,21,-0.000318,0.001456,-9.927584,0.107914,0.143048,848.368769,0.000164,0.000515,848.369448,0.319224


In [32]:
y=data.iloc[:,19]

In [33]:
y

0       0
1       0
2       0
3       0
4       0
       ..
8795    1
8796    1
8797    1
8798    1
8799    1
Name: label, Length: 8800, dtype: int64

In [34]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

In [35]:
from sklearn.ensemble import RandomForestClassifier


In [36]:
rfc=RandomForestClassifier(n_estimators=1000)


In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30)
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 1000)
classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000)

In [38]:
y_pred = classifier.predict(X_test)
y_pred

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [39]:
from sklearn.metrics import accuracy_score
print ("Accuracy : ", accuracy_score(y_test, y_pred)*100)

Accuracy :  72.91666666666666
